In [10]:
import pandas as pd
import plotly.express as px
from ipywidgets import Dropdown, Output, VBox
from IPython.display import display

# Load datasets
agri_data = pd.read_csv('Agriculture_final_2016.csv')
weather_data = pd.read_csv('weather.csv')

# Normalize state columns for case sensitivity and merge on these columns
agri_data['state_normalized'] = agri_data['State'].str.lower()
weather_data['state_normalized'] = weather_data['Station.State'].str.lower()

data = pd.merge(agri_data, weather_data, left_on='state_normalized', right_on='state_normalized')

# Initialize the output area for the plot
plot_output = Output()

# Function to update the plot based on the selected state
def update_plot(selected_state):
    plot_output.clear_output(wait=True)  # Clear the previous plot if any
    filtered_data = data[data['state_normalized'] == selected_state]
    fig = px.scatter(filtered_data, 
                     x='Data.Temperature.Avg Temp', 
                     y=' PRODUCTION, MEASURED IN $', 
                     color='Commodity',  # Different crops are colored differently
                     title=f'Impact of Sunlight on Crop Yields in {selected_state.title()}')
    with plot_output:
        fig.show()

# Dropdown to select the state
state_dropdown = Dropdown(options=sorted(data['state_normalized'].unique()))

# Function to handle dropdown changes
def on_state_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_plot(change.new)

state_dropdown.observe(on_state_change, names='value')

# Display the dropdown and output area
display(VBox([state_dropdown, plot_output]))

# Initialize the plot with the first state in the dropdown
update_plot(state_dropdown.options[0])
